In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

In [2]:
matplotlib.rcParams["figure.figsize"]=(20,10)
%matplotlib inline

In [3]:
address='/content/drive/MyDrive/ColabNotebooks/KPMG Virtual Internship/data quality/kpmgdata.xlsx'

#Transactions table

In [ ]:
t=pd.read_excel(address,sheet_name='Transactions',header=1)

In [ ]:
t

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,2003-07-21
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-10
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,2010-08-20
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-17


In [ ]:
t.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date'],
      dtype='object')

#finding potential profit

In [ ]:
t['profit']=t['list_price']-t['standard_cost']

In [ ]:
t.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02,17.87
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03,1702.55
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20,1544.61
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16,817.36
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10,1055.82


In [ ]:
t.to_csv('Transactions.csv',index=False)

#CustomerDemographics

In [4]:
c=pd.read_excel(address,sheet_name='CustomerDemographic',header=1)

In [5]:
c.columns

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object')

In [6]:
c.gender.unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

#dealing with gender column

In [7]:
def g(w):
  if w in ['Female','Femal','F']:
    return 'F'
  elif w in ['Male','M']:
    return 'M'
  else:
    return w

In [8]:
c['sex']=c.gender.apply(g)

In [9]:
c.sex.unique()

array(['F', 'M', 'U'], dtype=object)

In [10]:
c.drop('gender',axis=1,inplace=True)

#making date of birth null for customer id 34

In [11]:
c[c.DOB.dt.year==1843]

,customer_id,first_name,last_name,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,sex
33,34,Jephthah,Bachmann,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,NaN,No,20.0,U


In [12]:
c.at[33,'DOB']=np.NaN 

In [13]:
c[c.customer_id==34]

,customer_id,first_name,last_name,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,sex
33,34,Jephthah,Bachmann,59,NaT,Legal Assistant,IT,Affluent Customer,N,NaN,No,20.0,U


#dropping default column

In [14]:
c.drop('default',axis=1,inplace=True)

#finding ages of customers as of 1st jan 2017

In [15]:
from datetime import datetime,date
from dateutil.relativedelta import relativedelta

In [16]:
d=date(2017,1,1)
d

datetime.date(2017, 1, 1)

In [17]:
def func(x):
  try:
    return relativedelta(d,x).years
  except:
    return x

In [18]:
c['age']=c.DOB.apply(func)

In [19]:
c.columns

Index(['customer_id', 'first_name', 'last_name',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'sex', 'age'],
      dtype='object')

#making DOB and age null for rows where age is almost equal to tenure

In [20]:
c[c['age']<c['tenure']]

,customer_id,first_name,last_name,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,sex,age
1520,1521,Pernell,Duffett,49,1995-11-13,Business Systems Development Analyst,Financial Services,Affluent Customer,N,Yes,22.0,M,21
2053,2054,Horace,Gilpillan,12,1995-08-21,NaN,Manufacturing,Mass Customer,N,Yes,22.0,M,21


In [21]:
c.at[1520,'DOB']=np.NaN
c.at[2053,'DOB']=np.NaN
c.at[1520,'age']=np.NaN
c.at[2053,'age']=np.NaN

In [22]:
c[c.customer_id==1521]

,customer_id,first_name,last_name,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,sex,age
1520,1521,Pernell,Duffett,49,NaT,Business Systems Development Analyst,Financial Services,Affluent Customer,N,Yes,22.0,M,NaN


In [23]:
c[c.customer_id==2054]

,customer_id,first_name,last_name,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,sex,age
2053,2054,Horace,Gilpillan,12,NaT,NaN,Manufacturing,Mass Customer,N,Yes,22.0,M,NaN


In [24]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   past_3_years_bike_related_purchases  4000 non-null   int64         
 4   DOB                                  3910 non-null   datetime64[ns]
 5   job_title                            3494 non-null   object        
 6   job_industry_category                3344 non-null   object        
 7   wealth_segment                       4000 non-null   object        
 8   deceased_indicator                   4000 non-null   object        
 9   owns_car                             4000 non-null   object        
 10  tenure      

In [25]:
c.isna().sum()

customer_id                              0
first_name                               0
last_name                              125
past_3_years_bike_related_purchases      0
DOB                                     90
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                  87
sex                                      0
age                                     90
dtype: int64

In [ ]:
c.job_industry_category.unique()

array(['Health', 'Financial Services', 'Property', 'IT', nan, 'Retail',
       'Argiculture', 'Manufacturing', 'Telecommunications',
       'Entertainment'], dtype=object)

In [ ]:
c.to_csv('CustomerDemographic.csv',index=False)

#Address

In [ ]:
a=pd.read_excel(address,sheet_name='CustomerAddress',header=1)

In [ ]:
a.columns

Index(['customer_id', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')

In [ ]:
a.state.unique()

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [ ]:
a.address.value_counts()

3 Mariners Cove Terrace      2
3 Talisman Place             2
64 Macpherson Junction       2
359 Briar Crest Road         1
4543 Service Terrace         1
                            ..
5063 Shopko Pass             1
09 Hagan Pass                1
87897 Lighthouse Bay Pass    1
294 Lawn Junction            1
320 Acker Drive              1
Name: address, Length: 3996, dtype: int64

In [ ]:
a[a.address=='3 Mariners Cove Terrace']

,customer_id,address,postcode,state,country,property_valuation
2328,2333,3 Mariners Cove Terrace,3108,VIC,Australia,10
2980,2985,3 Mariners Cove Terrace,2216,NSW,Australia,10


In [ ]:
a[a.address=='64 Macpherson Junction']

,customer_id,address,postcode,state,country,property_valuation
2315,2320,64 Macpherson Junction,2208,NSW,Australia,11
3535,3540,64 Macpherson Junction,4061,QLD,Australia,8


In [ ]:
a[a.address=='3 Talisman Place']

,customer_id,address,postcode,state,country,property_valuation
732,737,3 Talisman Place,4811,QLD,Australia,2
2470,2475,3 Talisman Place,4017,QLD,Australia,5


#mapping names of states

In [ ]:
a.state.unique()

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [ ]:
def m(p):
  if p in ['New South Wales','NSW']:
    return 'NSW'
  elif p in ['VIC','Victoria']:
    return 'VIC'
  else:
    return p 

In [ ]:
a['state']=a.state.apply(m)

In [ ]:
a.state.unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [ ]:
a.to_csv('CustomerAddress.csv',index=False)